In [2]:
# Public
import pandas as pd
import numpy as np
import os, ast
import matplotlib.pyplot as plt
import pandas as pd
import warnings

warnings.filterwarnings('ignore', message='All-NaN slice encountered')

# Private
import signals
from functions import *
import cvxpy as cvx

## Load Data

In [9]:
##########################################################################################

### 이번에 바뀐것
import PortfolioAnalysis

strat_file = "/Users/kennypark/Desktop/quant_trading/Analysis/results/240115_vanilla_select_handpicked.csv"
ohlcv_path =  "/Users/kennypark/Desktop/quant_trading/Analysis/data/test_data"

# 요로케 써도되고
port1 = PortfolioAnalysis.Portfolio(strat_file, ohlcv_path)

# 요로케 써도됨. In case you want to make changes to df_strat, such as sorting out longs
df_strat = pd.read_csv(strat_file,index_col=0)
port1 = PortfolioAnalysis.Portfolio(df_strat, ohlcv_path) 

# If you want to access daily_lrets_df
port1_lrets_df = port1.get_daily_lrets(start_date = "2023-12-20", end_date="2024-01-10")

##########################################################################################

In [3]:
### Loads list of strategies we'll use ###
df = pd.read_csv("selected_signals_231229.csv",index_col=0)

In [27]:
df1 = df.loc[460]
df2 = df.loc[499]

combined_df = pd.concat([df1, df2], axis=0)

combined_df

strat_name            1000SHIBUSDT_15m_long_-5_[3, 0, 5]_[Asia/Hong_...
file                                 1000SHIBUSDT_15m_210829_231029.csv
symbol                                                     1000SHIBUSDT
timeframe                                                           15m
strategy                                                           blgr
threshold                                                            -5
cross_dir                                                          down
side                                                               long
barrier                                                       [3, 0, 5]
leverage                                                        1.85354
macro_tf                                                            all
macro_threshold                                                     NaN
macro_direction                                                     NaN
timezone                                                 Asia/Ho

In [67]:
lrets1 = get_lrets(df.loc[158], "data/2yrs")
lrets2= get_lrets(df.loc[921], "data/2yrs")

daily_lrets1 = lrets1.resample('D').sum()
daily_lrets1 = daily_lrets1[daily_lrets1 != 0]
n1 = 1#len(daily_lrets1)/2 

daily_lrets2 = lrets2.resample('D').sum()
daily_lrets2 = daily_lrets2[daily_lrets2 != 0]
n2 = 1#len(daily_lrets2)/2

mean_daily_lrets1 = daily_lrets1.mean()
mean_daily_lrets2 = daily_lrets2.mean()

std_daily_lrets1 = daily_lrets1.std()
std_lrets2 = daily_lrets2.std()

sharpe1 = mean_daily_lrets1/(std_daily_lrets1/np.sqrt(n1))
sharpe2 = mean_daily_lrets2/(std_lrets2/np.sqrt(n2))


cov = daily_lrets1.cov(daily_lrets2)

w1, w2 = 0.5, 0.5
combined_mean = (w1*mean_daily_lrets1+w2*mean_daily_lrets2)
combined_var = (w1*std_daily_lrets1)**2 + (w2*std_lrets2)**2 + 2*(w1*std_daily_lrets1)*(w2*std_lrets2)*cov
combined_n = 1#n1+n2
combined_sharpe = combined_mean/((combined_var**0.5)/np.sqrt(combined_n))

print(sharpe1)
print(sharpe2)
print(combined_sharpe)


# Calculated 
combined_lrets = pd.concat([lrets1, lrets2])
combined_daily_lrets =combined_lrets.resample('D').sum()
combined_daily_lrets = combined_daily_lrets[combined_daily_lrets!=0]
combined_n_manual = 1 #len(combined_daily_lrets)
combined_sharpe_manual = (combined_daily_lrets.mean()/(combined_daily_lrets.std()/np.sqrt(combined_n_manual)))

print(combined_sharpe_manual)


-0.09563378489331446
0.23536327865267964
0.07033650402909127
0.06747270817351675


In [68]:
0.5*mean_daily_lrets1+mean_daily_lrets2*0.5

0.0014722153551944134

In [69]:
combined_daily_lrets.mean()

0.0019891818033642263

In [37]:
print(calc_sharpe(daily_lrets1))
print(calc_sharpe(daily_lrets2))

-0.6566344666172086
1.7525169006175036


In [44]:
# Trade and returns processing function
def process_data(data_dir, start_date, timezone):
    end_date = os.listdir(data_dir)[0].replace(".", "_").split("_")[3]
    full_date_range = pd.date_range(start=pd.to_datetime(start_date).tz_localize(timezone), 
                                    end=pd.to_datetime(end_date, format='%y%m%d').tz_localize(timezone), 
                                    freq='D', tz=timezone)
    daily_lrets_df = pd.DataFrame(index=full_date_range)
    trades_df = pd.DataFrame()

    for index, row in df.iterrows():
        df_result = get_trade_history(df.loc[index], data_dir)
        df_result['strat_num'] = row['strat_num']
        df_result['exp_lret_per_24h'] = row['exp_lret_per_24h']

        lrets = df_result.lrets
        lrets.index = df_result.entry_time
        lrets.index = lrets.index.tz_convert(timezone)
        daily_lrets = lrets.resample('D').sum()
        daily_lrets_df[index] = daily_lrets

        trades_df = pd.concat([trades_df, df_result], ignore_index=True)

    daily_rets_df = np.exp(daily_lrets_df) - 1
    trades_df = trades_df[trades_df['entry_time'] >= pd.to_datetime(start_date).tz_localize(timezone)]
    trades_df = trades_df.sort_values(by="entry_time_actual")

    return daily_rets_df, trades_df

### Run the below only once

In [45]:
timezone = 'Asia/Seoul'

# # Process Train Data
data_train = 'data/2yrs'
start_date_train = "2022-10-01"
daily_rets_train, trades_train = process_data(data_train, start_date_train, timezone)

# Process Test Data
data_test = 'data/test_data'
start_date_test = "2023-10-01"
daily_rets_test, trades_test = process_data(data_test, start_date_test, timezone)

### Utilities

In [46]:
#########################################################################
def calc_pos_size(risk_perc, ent_price, sl_price, risked_tot_balance, leverage, fee_percent):
    potential_loss = abs(ent_price - sl_price)
    position_size0 = (risked_tot_balance * risk_perc / 100) / potential_loss 
    position_size = position_size0 / ( leverage * ( 1 - fee_percent) )
    return position_size
#########################################################################

#########################################################################
def solve_QP(q, C, k, min_weights):
    # dimension
    n = len(q)
    # Opt Var
    x = cvx.Variable(n)
    # Obj
    cost = q.T @ x + cvx.quad_form(x, -k * C)
    obj = cvx.Maximize(cost)
    # Const
    const = [cvx.sum(x) == 1, x >= min_weights]
    # QP
    prob = cvx.Problem(obj, const)
    try:
        prob.solve()
        if prob.status == 'optimal':
            return x.value, prob.value
        else:
            raise ValueError("Optimization failed")
    except Exception as e:
        raise ValueError(f"Optimization failed: {e}")
#########################################################################

#########################################################################
def optimize_portfolio_cvar(daily_rets_df, min_weight_threshhold, alpha=0.95):

    n_assets = daily_rets_df.values.shape[1]
    cost = -daily_rets_df.values 
    
    # Portfolio weights
    weights = cp.Variable(n_assets)

    # VaR (value-at-risk) as an auxiliary variable
    VaR = cp.Variable()

    excess_losses = cp.Variable(cost.shape[0])

    # Portfolio loss for each day
    portfolio_cost = cost @ weights

    # Constraints;
    constraints = [
        cp.sum(weights) == 1,
        weights >= min_weight_threshhold,
        excess_losses >= 0,
        excess_losses >= portfolio_cost - VaR, 
    ]

    # Obj; minimizing the CVaR (VaR + average excess losses)
    num_days = cost.shape[0]
    CVaR = VaR + cp.sum(excess_losses) / (num_days * (1 - alpha))

    # QP
    objective = cp.Minimize(CVaR)
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cvx.ECOS)

    # Optimal weights
    return weights.value
#########################################################################
    
#########################################################################
def monthly_eval(portfolio_lrets):
    monthly_lrets = portfolio_lrets.groupby([portfolio_lrets.index.year, portfolio_lrets.index.month]).sum()
    monthly_rets = np.exp(monthly_lrets)-1

    # Monthly Sharpe
    monthly_sharpe_ratios = portfolio_lrets.groupby([portfolio_lrets.index.year, portfolio_lrets.index.month]).apply(calc_sharpe)

    # Monthly MDD
    monthly_mdd = abs(portfolio_lrets.groupby([portfolio_lrets.index.year, portfolio_lrets.index.month]).apply(get_mdd))

    # Monthly Calmar (=Lret/MDD)
    monthly_calm = monthly_lrets/monthly_mdd

    return {
        "Monthly Total Return": monthly_rets,
        "Monthly Sharpe Ratios": monthly_sharpe_ratios,
        "Monthly Max Drawdown": monthly_mdd,
        "Monthly Calmar Ratio": monthly_calm
    }
#########################################################################

#########################################################################
def print_eval_results(results):
    for metric, data in results.items():
        print(f"\n{metric}:\n{'-' * len(metric)}")
        if isinstance(data, pd.DataFrame) or isinstance(data, pd.Series):
            print(data.to_string())
        else:
            print(data)
#########################################################################

### QP Weighting

In [47]:
daily_rets_train = daily_rets_train.fillna(0)
q = daily_rets_train.mean().values
q = daily_rets_train.ewm(span=20).mean().iloc[-1].values
C = daily_rets_train.cov().values

# ------- Randomly selected parameters
k = 600 # tune this parameter (currently set at random) - it's called the risk aversion parameter
min_weight_threshhold = 0.0000
solution, obj_value = solve_QP(q, C, k, min_weight_threshhold)
rounded_solution = np.where(np.abs(solution) < 1.0e-10, 0.0, np.round(solution, decimals=5))

# print("\nOptimal solution:\n", rounded_solution)
print("\nObjective value:", obj_value)


# use test set
weights = rounded_solution
weighted_daily_lrets_df = np.log(daily_rets_test*weights+1)
portfolio_lrets = weighted_daily_lrets_df.sum(axis=1)

pf_perf = monthly_eval(portfolio_lrets)
print("\n\n------------- Mean-Variance QP -------------")
print_eval_results(pf_perf)


Objective value: 0.0014216150000800518


------------- Mean-Variance QP -------------

Monthly Total Return:
--------------------
2023  10    0.052039
      11    0.024312
      12   -0.001437

Monthly Sharpe Ratios:
---------------------
2023  10    12.500231
      11     7.491622
      12    -0.386931

Monthly Max Drawdown:
--------------------
2023  10    0.002217
      11    0.004819
      12    0.011523

Monthly Calmar Ratio:
--------------------
2023  10    22.884485
      11     4.984986
      12    -0.124788


# CVaR Weighting

In [50]:
# Using training set
optimal_weights = optimize_portfolio_cvar(daily_rets_train, min_weight_threshhold = 0.01, alpha = 0.80)

# Using testing set
weighted_daily_lrets_df = np.log(daily_rets_test*optimal_weights+1)
portfolio_lrets = weighted_daily_lrets_df.sum(axis=1)


pf_perf = monthly_eval(portfolio_lrets)
print("\n\n------------- CVaR -------------")
print_eval_results(pf_perf)

NameError: name 'cp' is not defined

In [51]:
weights = 0.01
weighted_daily_lrets_df = np.log(daily_rets_test*weights + 1)
portfolio_lrets = weighted_daily_lrets_df.sum(axis=1)

pf_perf = monthly_eval(portfolio_lrets)
print("\n\n------------- Equal Weights -------------")
print_eval_results(pf_perf)



------------- Equal Weights -------------

Monthly Total Return:
--------------------
2023  10    0.011618
      11    0.012623
      12    0.002033

Monthly Sharpe Ratios:
---------------------
2023  10    8.460178
      11    9.502156
      12    0.967175

Monthly Max Drawdown:
--------------------
2023  10    0.002915
      11    0.001552
      12    0.007844

Monthly Calmar Ratio:
--------------------
2023  10    3.963317
      11    8.082230
      12    0.258908
